<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/Voice%20Processing/Lab%201/Prelab_voice_proc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Αϊδίνης Γιώργος  (el16031)

> Τσιλιβής Θοδωρής (el16032)



---
# 1η Εργαστηριακή Άσκηση - Μέρος πρώτο: Ορθογράφος

Πριν αρχίσουμε τα βήματα για την προετοιμασία του εργαστηρίου, πρέπει να εγκαταστήσουμε κάποια εργαλεία που θα χρησιμοποιήσουμε στην εργασία αυτή. Πρώτα είναι η βιβλιοθήκη OpenFST (1.6.1), για την οποία χρησιμοποιούμε το έτοιμο bash script.

In [96]:
!chmod +x install_openfst.sh
!./install_openfst.sh

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'zlib1g-dev' instead of 'libz-dev'
make is already the newest version (4.1-9.1ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
checkinstall is already the newest version (1.6.2-4ubuntu2).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
wget is already the newest version (1.19.4-1ubuntu2.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
--2019-12-04 01:54:21--  http://www.openfst.org/twiki/pub/FST/FstDownload/openfst-1.6.1.tar.gz
Resolving www.openfst.org (www.openfst.org)... 206.196.111.47
Connecting to www.openfst.org (www.openfst.org)|206.196.111.47|:80... connected.
HTTP 

Ακόμα, θα πρέπει να εγκαταστήσουμε τις βιβλιοθήκες του NLTK (Natural Language Toolkit) καθώς και του graphviz, που μας επιτρέπει να σχεδιάζουμε τα FST που δημιουργούμε.

In [97]:
!pip install nltk
!sudo apt install python-pydot python-pydot-ng graphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.40.1-2).
python-pydot is already the newest version (1.2.3-1).
python-pydot-ng is already the newest version (1.0.0-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


---
# Βήμα 1ο

Για το Βήμα αυτό, επιλέξαμε και τα δύο προτεινόμενα βιβλία από το Project Guternberg, για να έχουμε περισσότερα δεδομένα.

Το τελικό αρχείο, ονομάζεται corpora.txt . 

Ιδανικά, θα θέλαμε να είχαμε όσο το δυνατόν περισσοτερα δεδομενα, καθως αυτο θα μας επετρεπε να εχουμε ευρυτερο λεξιλογιο, δηλαδη να αναγνωριζει το συστημα μας περισσοτερες λεξεις. Ακομα, θα μπορουσαμε να εχουμε στην διαθεση μας διαφορετικες γραφες των ιδιων λεξεων, που ειναι χρησιμο σε εξειδικευμενους ορθογραφους.

Προσδιοριζουμε την κωδικοποιηση χαρακτηρων σε UTF-8, για να αναγνωρισθουν τυχον "περιεργοι" χαρακτηρες.

Ακομα, ειναι χρησιμο να κανουμε import τωρα ο,τι βιβλιοθηκες της python θα χρησιμοποιησουμε αργοτερα:

In [0]:
# -*- coding: utf-8 -*-


import itertools
from nltk.tokenize.regexp import regexp_tokenize

---
# Βήμα 2ο



> α)  Γραφουμε μια συναρτηση που την ονομαζουμε identity_preprocess: (αρκετα απλο)




In [0]:
"""Reads a string and returns it. Returns string."""
def identity_preprocess(stringaki):
	return stringaki




> β) Γραφουμε μια συναρτηση που να δεχεται δυο ορισματα, το ονομα αρχειου, και μια συναρτηση της επιλογης μας και να διαβαζει το αρχειο γραμμη - γραμμη καλωντας την συναρτηση που εχουμε δωσει σαν ορισμα σε καθε γραμμη:

In [0]:
"""Takes 2 arguments, one being the filepath and the second
being the name of the method that reads the string, defaulting
to the above method, 'identity_preprocess'. Returns a list of
strings, that are the whole text of the file."""
def read_from_file(filepath, myfunc = identity_preprocess):
	lines = []
	with open(filepath) as file:
		for line in file:
			lines.append(myfunc(line))
	return lines

> γ) Γραφουμε εδω 2 συναρτησεις, η μια ειναι η συναρτηση clean_text που "καθαριζει" το κειμενο που της δινουμε απο οποιον χαρακτηρα δεν ειναι γραμμα, και στην συνεχεια, την tokenize, που παιρνει ενα τετοιο καθαρο κειμενο και το χωριζει σε μερη:

In [0]:
"""Takes one string and returns a string that has only lowercase
letters, that means exluding any non-alphanumeric sumbols, numbers
punctuation."""
def clean_text(text):
	#replace new line and carriage return
	text = text.replace("\n", " ").replace("\r", " ")
	#replace the numbers, symbols and punctuation with space
	punc_list = '~`!@#$£½%^&*()_+-—“”’‘\'=}{|:à"?>æ<âç,œ./è;é][}' + '1234567890'
	t = str.maketrans(dict.fromkeys(punc_list, " "))
	text = text.translate(t)
	#make everything lowercase
	text = text.lower()
	return text



"""Takes a text and splits it between whitespaces or \n. Returns
a list of strings"""
def tokenize(text):
	text = clean_text(text)
	text = text.split()
	return text




> δ) μια καλη συναρτηση που βρηκαμε οτι εχει καλα αποτελεσματα ειναι η εξης:



In [0]:

"""Uses the imported library from nltk to return a list of strings
created by using regular expressions. It is one of the fastest 
tokenizers."""
def Tokenize(text):
	#text = clean_text(text)
	return regexp_tokenize(text, pattern = '\s+', gaps = True)


Στις προσπαθειες μας, βρηκαμε οτι η ετοιμη συναρτηση ενω ειναι παρα πολυ γρηγορη, (πολυ πιο γρηγορη απο την δικη μας, ακομα και για τον σχετικα μικρο αριθμο δεδομενων μας), επιτρεπει λιγο λιγοτερες λεξεις (~170k vs ~173k) να "περασουν" ομως καταληγουμε με πολυ μεγαλο λεξιλογιο (κατι που θα φανει μετεπειτα): (παραθετουμε και την συναρτηση flatten, που αν και δεν ζητειται, ειναι χρησιμη καθως μετατρεπει μια λιστα λιστων σε μια λιστα)

In [103]:
"""Takes a list of lists and returns a list. Elements are kept in
order."""
def flatten(l):
	return list(itertools.chain(*l))
 

lines = read_from_file("corpora.txt")
text = []
text1 = []
for line in lines:
    text.append(Tokenize(line))
    text1.append(tokenize(line))
text = flatten(text)
text1 = flatten(text1)
print(len(text))
print(len(text1))

170792
173206


---
# Βήμα 3ο



> α)  Για να βρουμε το λεξιλογιο μας, φτιαχνουμε την συναρτηση unique_tokens, που παιρνει μια λιστα απο strings και αφαιρει τις πολλαπλες εμφανισεις:

In [0]:
"""Takes a list of strings (tokens) and reduces it to only unique
elements. (gets rid of the duplicates). Returns a list of strings
that are unique and sorted."""
def unique_tokens(tokens):
	lexicon = list(set(tokens))
	lexicon.sort()
	return lexicon

> β)  Για να βρουμε το αλφαβητο του corpus, φτιαχνουμε την συναρτηση alphabet, που παιρνει μια λιστα απο strings επιστρεφει μια λιστα απο χαρακτηρες, χωρις πολλαπλες εμφανισεις:

Εκτελωντας τις παρακατω εντολες, μπορουμε να δουμε οτι τα λεξιλογια που δημιουργουνται απο τις διαφορετικες μεθοδους tokenization ειναι κατα πολυ διαφορετικα:

In [105]:
lines = read_from_file("corpora.txt")
text = []
for line in lines:
    text.append(Tokenize(line))
text = flatten(text)
Lexicon = unique_tokens(text)
print("NLTK tokenize(): ",len(Lexicon))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
Lexicon = unique_tokens(text)
print("Our tokenize(): ",len(Lexicon))

NLTK tokenize():  21672
Our tokenize():  11249


In [0]:
"""Takes a string and returns a list of the characters 
that were used to make the strings."""
def split_word(word):
	return [char for char in word]



"""Takes a list of strings as an input and returns a list of chars.
The characters that are returned are used to make the elements of 
the input strings, but they are returned duplicate-free and sorted."""
def alphabet(words):
	chars = []
	for word in words:
		chars.append(split_word(word))
	alpbt = list(set(flatten(chars)))
	alpbt.sort()
	return alpbt

---
# Βήμα 4ο


Για να δημιουργησουμε το αρχειο chars.syms που μας χρειαζεται για μετα, φιταχνουμε δυο συναρτησεις, μια για να κανει το (προηγουμενως δημιουργημενο) αλφαβητο indexed, δηλαδη καθε χαρακτηρας να αντιστοιχιζεται σε εναν φυσικο αριθμο, και το ε ('<epsilon>') να αντιστοιχιζεται στο 0. Ακομα, φτιαχνουμε μια συναρτηση που να παιρνει ενα τετοιο στοιχισμενο αλφαβητο και να το αποθηκευει σε ενα αρχειο της επιλογης μας (προεπιλεγμενο ονομα ειναι το "chars.syms"):


In [0]:
"""Takes an alphabet (list of chars) and assigns each character 
an index. Index 0 is <epsilon> (ε). Returns a list that has n+1 
tuples, each tuple being the character with its assigned index."""
def alphabet_indexing(alphabet):
	indexed_alphabet = []
	indexed_alphabet.append(("<epsilon>", 0))
	i = 1;
	for letter in alphabet:
		indexed_alphabet.append((letter, str(i)))
		i+=1
	return indexed_alphabet



"""Takes two arguments. The first is an  alphabet and the 
second is the name of the file to write the output. The 
method takes the indexed alphabet and writes it to a file 
with the name given"""
def symbols_file(alphabet, filename = "chars.syms"):
	f = open(filename, "w+")
	for pair in alphabet:
		f.write(pair[0] + "\t\t" + str(pair[1]) + "\n")


---
# Βήμα 5ο


   α) Για to βημα 5ο, αρχικα υποθεσαμε οτι πρεπει να υλοποιησουμε εκτος των αλλων και την συναρτηση για την αποσταση Levenshtein, οποτε αν και (μετα απο ξεκαθαρισμα) δεν ειναι απαραιτητο, την παραθετουμε εδω για κριτικη, καθως και για να υπαρχει σε τυχον περιπτωση που την χρειαστουμε:

In [0]:
"""This function computes the Levenshtein distance between two 
chars s,t. It uses the algorithm mentioned in the instructions
of the lab, and is horribly inefficient. Takes two arguments, both
chars and returns an integer that is the levenshtein distance of
the two strings."""
def my_levenshtein(s,t):
	#same characters, no edit => Levenshtein cost 0
	if (s==t):
		return 0
	#<epsilon> with some other character or any character with <epsilon>
	#either insertion or deletion => Levenshtein cost 1
	elif (s=='<epsilon' and t!='<epsilon>') or (s!='<epsilon' and t=='<epsilon>'):
		return 1
	#two different chars, substiturion => Levenshtein cost 1
	else:
		return 1



"""This function computes the Levenshtein distance between two 
strings s,t. It uses the iterative algorithm, that creates a 
2d matrix of the characters of the strings, along with the 
values it needs to compute the sum of the weights of the insert
/delete/substitute logic. It is a much faster algorithm than the
recursive one. The method takes 3 arguments, string s, string t
and a tuple of 3 integeres that are the cost of deletion, insertion
and substitution respectively. Defaults to (1,1,1) (equal cost).
I have taken the code for this method from:
https://www.python-course.eu/levenshtein_distance.php 
Another method could have been the method given in
the python package python-Levenshtein (0.12.0) that uses python to 
instruct C to compute the afforementioned array, with much faster 
execution times."""
def iterative_levenshtein(s, t, costs=(1, 1, 1)):
    """ 
        iterative_levenshtein(s, t) -> ldist
        ldist is the Levenshtein distance between the strings 
        s and t.
        For all i and j, dist[i,j] will contain the Levenshtein 
        distance between the first i characters of s and the 
        first j characters of t
        
        costs: a tuple or a list with three integers (d, i, s)
               where d defines the costs for a deletion
                     i defines the costs for an insertion and
                     s defines the costs for a substitution
    """
    rows = len(s)+1
    cols = len(t)+1
    deletes, inserts, substitutes = costs
    
    dist = [[0 for x in range(cols)] for x in range(rows)]
    # source prefixes can be transformed into empty strings 
    # by deletions:
    for row in range(1, rows):
        dist[row][0] = row * deletes
    # target prefixes can be created from an empty source string
    # by inserting the characters
    for col in range(1, cols):
        dist[0][col] = col * inserts
        
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0
            else:
                cost = substitutes
            dist[row][col] = min(dist[row-1][col] + deletes,
                                 dist[row][col-1] + inserts,
                                 dist[row-1][col-1] + cost) # substitution
    """for r in range(rows):
        print(dist[r])
    """ 
    return dist[row][col]

Προφανως η ασκηση δεν ανεφερε αυτο, οποτε για την δημιουργια του μετατροπεα μιας καταστασης, αρχικα δημιουργουμε τα αρχεια που χρειαζομαστε, (τα αρχεια για τα συμβολα ειναι ηδη ετοιμα, καθως ειναι ομοια με τα αρχεια του στοιχισμενου αλφαβητου που περιεχεται στο αρχειο chars.syms. Για τον σκοπο αυτο, φτιαξαμε μια μεθοδο για να τυπωνει τα περιεχομενα του fst τοσο στο stdout, οσο και στο αρχειο με ονομα "text.txt" που θα χρησιμοποιησουμε αργοτερα. O ακριβης αριθμος των ακμων στο fst ειναι (πληθος αλφαβητου)^2 -1, καθως θελουμε καθε γραμμα να εχει καποια ενεργεια με οποιοδηποτε αλλο γραμμα του αλφαβητου, εκτος απο το ε, που με τον εαυτο του δεν θελουμε να εχει. 

In [0]:
def text_fst(Alphabet, filename = "text.txt", print_lines = False):
    f = open(filename, "w+")
    if (print_lines):
        for pair in Alphabet:
            for pair1 in Alphabet:
                if  (pair[0] == pair1[0] =="<epsilon>"):
                    continue 
                elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 0")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
                elif (pair[0]=='<epsilon>'):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 1")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                elif (pair1[0]=='<epsilon>'):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 1")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                else:
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 1")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
        print("0")
        f.write("0")
    else:
        for pair in Alphabet:
            for pair1 in Alphabet:
                if  (pair[0] == pair1[0] =="<epsilon>"):
                    continue 
                elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
                elif (pair[0]=='<epsilon>'):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                elif (pair1[0]=='<epsilon>'):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
                else:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
        f.write("0")
    f.close()


Για να δημιουργησουμε τα αρχεια, θα πρεπει να εκτελεσουμε τον παρακατω κωδικα:

In [0]:
lines = read_from_file("corpora.txt")
#print(len(lines))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
#print(len(text))
Lexicon = unique_tokens(text)
#print(len(Lexicon))
Alphabet = alphabet(Lexicon)
#print(len(Alphabet))
Alphabet = alphabet_indexing(Alphabet)
#print(Alphabet)
symbols_file(Alphabet,)
text_fst(Alphabet, "onestate.txt")

Στην συνεχεια, για να δημιουργησουμε το fst, θα πρεπει να εκτελεστει το παρακτω shell script:

In [111]:
!fstcompile -isymbols=chars.syms -osymbols=chars.syms onestate.txt > onestate.fst
!fstinfo onestate.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       1
# of arcs                                         728
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               26
# of output epsilons                              26
input label multiplicity                          26.9643
output label multiplicity                         26.9643
# of accessible states                            1
# of coaccessible states                          1
# of connected states                             1
# of connected components                         1
# of strongly conn components                     1
input matcher                 

Η πρωτη εντολη κανει compile τις πληροφοριες του αρχειο που δημιουργησαμε προηγουμενως για το one state fst, και παιρνει σαν input και output symbols τους χαρακτηρες του chars.syms (που ειναι και η αλφαβητα μας).
Η δευτερη εντολη σχεδιαζει το fst που δημιουργηθηκε με τις βιβλιοθηκες dot και ghraphviz, και το αποθηκευει στο αρχειο fst.jpg. Η τριτη εντολη ειναι για να δουμε οτι ολα πηγαν καλα, δηλαδη οτι δημιουργηθηκε και εχει τα καταλληλα χαρακτηρηστικα. Επειδη ο μετατροπεας μιας καταστασης για την περιπτωση μας ειναι εξαιρετικα μεγαλος και η εικονα (jpg) δεν μας δειχνει καθαρα το τι συμβαινει, θα κανουμε μια μικροτερη αποτυπωση, απλα και μονο για λογους κατανοησης της λειτουργιας του κωδικα:

In [0]:
Alphabet = [('<epsilon>', 0), ('a', '1'), ('b', '2'), ('c', '3'), ('d', '4')]
symbols_file(Alphabet, "chars1.syms")
text_fst(Alphabet, "text1.txt", False)

Τα αρχεια αυτα εχουν 25 ακμες, οποτε ειναι αρκετα πιο ευκολο να διακρινουμε τι γινεται απο την εικονα:

In [113]:
!fstcompile -isymbols=chars1.syms -osymbols=chars1.syms text1.txt > fst1.fst
!fstdraw --isymbols=chars1.syms --osymbols=chars1.syms -portrait fst1.fst  | dot -Tjpg >fst1.jpg
!fstinfo fst1.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       1
# of arcs                                         24
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               4
# of output epsilons                              4
input label multiplicity                          4.83333
output label multiplicity                         4.83333
# of accessible states                            1
# of coaccessible states                          1
# of connected states                             1
# of connected components                         1
# of strongly conn components                     1
input matcher                    

Ο μετατροπεας εισοδου αν παρουμε το shortest path, αυτο σημαινει οτι δεν επιφερει καμια αλλαγη, δηλαδη η λεξη που παιρνει στην εισοδο και η λεξη που παιρνουμε στην εξοδο ειναι ιδιες.

   β) Ιδανικα, αντι να εχουμε κανονες για τα βαρη, θα επρεπε αν ειχαμε πολυ περισσοτερα δεδομενα να εφαρμοζουμε πιθανοτικες μεθοδους για να προσδιορισουμε τα βαρη καλυτερα στο μοντελο μας.

---
# Βήμα 6ο
   α) Για να υλοποιησουμε τον αποδοχεα λεξεων του λεξικου, ακολουθουμε την λογικη που ειχαμε και πριν για το αλφαβητο, μονο που αυτην την φορα ο αποδοχεας ειναι παρα (μα παρα) πολυ μεγαλος (τουλαχιστον για τον υπολογιστη μας). Παρολα αυτα, η μεθοδος sixth δημιουργει ενα αρχειο με το προσδιορισμενο ονομα, που περιεχει τις ακμες για καθε λεξη, με μηδενικο βαρος.

In [0]:
def sixth(Lexicon, Alphabet, filename = "6a.txt", print_lines = False):
    for i in range(0, len(Alphabet)):
        temp = Alphabet[i]
        Alphabet[i] = [temp, i]
    f = open(filename, "w+")
    i = 2
    if (not print_lines):
        for word in Lexicon:
            for j in range(0, len(word)):
                if j==0:
                    f.write("0" + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n") 
                    i += 1
                elif (j== len(word) -1):
                    f.write( str(i-1) + " " + "1" + " " + word[j] + " " + word[j] + " 0" + "\n")
                    i +=1
                else:
                    f.write( str(i-1) + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                    i+=1
        f.write("1")
    else:
        for word in Lexicon:
            for j in range(0, len(word)):
                if j==0:
                    f.write("0" + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                    print("0" + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n") 
                    i += 1
                elif (j== len(word) -1):
                    f.write( str(i-1) + " " + "1" + " " + word[j] + " " + word[j] + " 0" + "\n")
                    print( str(i-1) + " " + "1" + " " + word[j] + " " + word[j] + " 0" + "\n") 
                    i +=1
                else:
                    f.write( str(i-1) + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                    print( str(i-1) + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                    i+=1
        f.write("1")
        print("1")
    f.close()


In [0]:
lines = read_from_file("corpora.txt")
#print(len(lines))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
#print(len(text))
Lexicon = unique_tokens(text)
#print(len(Lexicon))
Alphabet = alphabet(Lexicon)
sixth(Lexicon, Alphabet, "acceptor.txt")


 Αφου φτιαξουμε το αρχειο, οπως πριν καλουμε απο το cli τις ιδιες εντολες με τον μετατροπεα μιας καταστασης (με αλλαγμενα τα ονοματα προφανως. Συστηνουμε (τουλαχιστον εμεις ετσι μπορουσαμε να δουμε τα αποτελεσματα) να εκτελεστει το απο κατω κελι αντι για το απο πανω, για και στην συνεχεια να δημιουργηθει το fst, γιατι μειωνει σημαντικα ( απο 11249 λεξεις σε μολις 10 ) το λεξιλογιο, οποτε οχι μονο ειναι γρηγορη η δημιουργια του fst, αλλα και μπορει να διακριθει ο ιδιος ο αποδοχεας.

In [116]:
lines = read_from_file("corpora.txt")
#print(len(lines))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
#print(len(text))
Lexicon = unique_tokens(text)
print("Αρχικο μεγεθος λεξικου: ",len(Lexicon))
Alphabet = alphabet(Lexicon)
Lexicon1 = Lexicon[10:20]
print("Μειωμενο μεγεθος λεξικου: ",len(Lexicon1))
sixth(Lexicon1, Alphabet)

Αρχικο μεγεθος λεξικου:  11249
Μειωμενο μεγεθος λεξικου:  10


In [117]:
!fstcompile -isymbols=chars.syms -osymbols=chars.syms acceptor.txt > acceptor.fst
!fstinfo acceptor.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       69650
# of arcs                                         80875
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          96.88
output label multiplicity                         96.88
# of accessible states                            69650
# of coaccessible states                          69628
# of connected states                             69628
# of connected components                         1
# of strongly conn components                     69650
input matcher 

   β) Για το μερος β, αρκει να τρεξουμε τις παρακατω εντολες (τις εχουμε βαλει ξεχωριστα για να παρατηρησουμε τι κανει η καθε μια)

In [118]:
!fstdeterminize acceptor.fst  > acceptor_determinized.fst
!fstminimize acceptor_determinized.fst > acceptor_minimized.fst
!fstinfo acceptor_minimized.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       7933
# of arcs                                         15422
initial state                                     0
# of final states                                 1101
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          1
output label multiplicity                         1
# of accessible states                            7933
# of coaccessible states                          7933
# of connected states                             7933
# of connected components                         1
# of strongly conn components                     7933
input matcher           

Η συναρτηση fstrmepsilon αφαιρει τις μεταβασεις που εχουν ε στο fst μας, και προφανως δεν ειχε καν νοημα να τις εφαρμοσουμε, αφου δεν ειχαμε χρησιμοποιησει καν ε στην κατασκευη του.
Η συναρτηση fstdeterminize αφαιρει τον μη ντετερμινισμο, και οπως φαινεται απο τις εικονες των διαγραμματων που παραγονται, αν και δεν εχει ελαχιστο αριθμο καταστασεων, ειναι αφενος σαφως μικροτερο διαγραμμα απο το αρχικο και αφετερου σαφως πιο κατανοητο.
Τελος, η συναρτηση fstminimize δημιουργει το διαγραμμα με τον ελαχιστο αριθμο καταστασεων, οπως φαινεται και απο την φωτογραφια fst_minimized.jpg που εχει 24 καταστασεις (ενω η determinized 27, ενω η αρχικη 55!).

*Για τα Βηματα 7,8,9, δεν ειχαμε αρκετο χρονο, καθως πολλες αποριες μας λυθηκαν την ημερα του εργαστηριου (25/11). Παρολα αυτα, οπως συζητηθηκε και με τον καθηγητη, θα ειναι υλοποιημενα στην υποβολη της ολοκληρωμενης αναφορας για το εργαστηριο 1.*

---
# Βήμα 7ο

   α) Φτιαχνουμε το Levenshtein Transducer:

In [119]:
!fstcompose onestate.fst acceptor_minimized.fst > Leven.fst
!fstinfo Leven.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       7933
# of arcs                                         622652
initial state                                     0
# of final states                                 1101
# of input/output epsilons                        0
# of input epsilons                               15422
# of output epsilons                              206258
input label multiplicity                          4.20416
output label multiplicity                         26.6687
# of accessible states                            7933
# of coaccessible states                          7933
# of connected states                             7933
# of connected components                         1
# of strongly conn components                     7933
in

Αναλογα με τα βαρη του καθε ειδους edit, γινεται αναλογο prioritization στα output words. Για παραδειγμα, για ισοβαρη edits, ο min distance transducer μπορει να επιλεξει και λεξεις μεγαλυτερου μηκους, που εχουν πολλα κοινα γραμματα με την αρχικη, με τελικο βαρος ισο με αλλες λεξεις μικροτερου μηκους, που ομως εχουν περισσοτερα substitutions γραμματων. Απο την αλλη, αν ειχαμε μεγαλυτερο βαρος στο removal η στο insertion, τοτε θα ειχαμε περισσοτερα αποτελεσματα ιδιου μηκους με την λεξη εισοδου, αλλα με περισσοτερα substitutions. Συμπερασματικα, καταληγουμε οτι τα βαρη επηρεαζουν την μορφη των λεξεων εξοδου. 

 β) Φτιαχνουμε τα απαραιτητα αρχεια για την δημιουργια ενος acceptor για την λεξη "cit":


In [0]:
Lexicon_cit = ["cit"]
sixth(Lexicon_cit, Alphabet, "cit.txt")

Φτιαχνουμε τον αποδοχεα για την λεξη "cit":

In [0]:
! fstcompile -isymbols=chars.syms -osymbols=chars.syms cit.txt > cit.fst
! fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait cit.fst  | dot -Tjpg >cit.jpg

Κανουμε compose το FST για την αποσταση Levenshtein με τον αποδοχεα της λεξης "cit":

In [122]:
! fstcompose cit.fst Leven.fst  > min_distance.fst
! fstinfo min_distance.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       31732
# of arcs                                         131753
initial state                                     0
# of final states                                 1101
# of input/output epsilons                        0
# of input epsilons                               61688
# of output epsilons                              23799
input label multiplicity                          4.05012
output label multiplicity                         1.70231
# of accessible states                            31732
# of coaccessible states                          31732
# of connected states                             31732
# of connected components                         1
# of strongly conn components                     3173

Καλουμε την fstshortestpath για να βρουμε την ελαχιστη edit αποσταση (αποσταση Levenshtein)

In [123]:
!fstshortestpath min_distance.fst | fstrmepsilon > out.fst
!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait out.fst  | dot -Tjpg >out.jpg
!fstinfo out.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       4
# of arcs                                         3
initial state                                     3
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          1
output label multiplicity                         1
# of accessible states                            4
# of coaccessible states                          4
# of connected states                             4
# of connected components                         1
# of strongly conn components                     4
input matcher                                 

Αν τρεξουμε τον παραπανω κωδικα, βλεπουμε οτι η προβλεψη του min edit spell checker ειναι η λεξη "wit" που ουσιαστικα εχει μονο ενα substitution ενος γραμματος. Αν βαλουμε το argument --nshorterst = Ν, οπου Ν ο αριθμος των προβλεψεων που θελουμε να δουμε, βλεπουμε οτι υπαρχουν πολλες επιλογες με το ιδιο αριθμο edits, οπως η λεξη "sit" και αλλες. 

---
# Βήμα 8ο

  α) Για το βημα αυτο, εχουμε αποηθεκυσει τα δεδομενα για testing στο αρχειο evaluation_data.txt .
  β) Για το ερωτημα αυτο, πρωτα δημιουργουμε ενα λεξικο που συνδεει τις σωστες λεξεις με τις λαθος, και απο αυτο "τραβαμε" τυχαια λαθος λεξεις και χρησιμοποιουμε τον min edit checker για να μας δωσει πιθανες απαντησεις λεξεων. Στην συνεχεια συγκρινουμε τις απαντησεις του με τις προκαθορισμενες απαντησεις, τυπωνοντας για καθε περιπτωση.

In [124]:
import random
lines = read_from_file("evaluation_data.txt")
text = []
dictionary = {}
for line in lines:
    text = []
    text.append(tokenize(line))
    for i in range (1,len(text[0])):
      dictionary.update({text[0][i]:text[0][0]})
for counter in range(0,20):
  key = random.choice(list(dictionary.keys()))
  value = dictionary[key]
  Lexicon_test = [key]
  f = open('new.txt', 'w+')
  f.truncate(0)
  f.close()
  fileopen = open("new.txt","w+")
  for i in range(0,len(key)):
    fileopen.write("%s " %str(i))
    fileopen.write(" %s " %str(i+1))
    fileopen.write(" %s " %key[i])
    fileopen.write(" %s " %key[i])
    fileopen.write("0\n")
  fileopen.write(str(len(key)))
  fileopen.close()
  sixth(Lexicon_test, Alphabet, "new.txt")
  !fstcompile -isymbols=chars.syms -osymbols=chars.syms new.txt > test.fst
  !fstcompose test.fst Leven.fst  > min_distance.fst
  print("Wrong word = " ,key,", actual word = ",value)
  print("What we got from the transducer =", end =" ")  
  !fstshortestpath min_distance.fst | fstrmepsilon | fsttopsort | fstprint -osymbols=chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n'
  print()

Wrong word =  magnificant , actual word =  magnificent
What we got from the transducer = magnificent
Wrong word =  muinets , actual word =  minutes
What we got from the transducer = mines
Wrong word =  desicate , actual word =  desiccate
What we got from the transducer = delicate
Wrong word =  adres , actual word =  address
What we got from the transducer = acres
Wrong word =  proplen , actual word =  problem
What we got from the transducer = problem
Wrong word =  questionaire , actual word =  questionnaire
What we got from the transducer = questionable
Wrong word =  juce , actual word =  juice
What we got from the transducer = june
Wrong word =  stomac , actual word =  stomach
What we got from the transducer = stomachs
Wrong word =  acomodation , actual word =  accommodation
What we got from the transducer = abomination
Wrong word =  benifits , actual word =  benefits
What we got from the transducer = benefit
Wrong word =  offten , actual word =  often
What we got from the transducer 

---
# Βήμα 9ο

  α) Διαβαζουμε το κειμενο σε προτασεις:

In [0]:
lines = read_from_file("corpora.txt")
text = []
for line in lines:
    text.append(tokenize(line))

  β) Κανουμε import την βιβλιοθηκη του Word2Vec, για να εκπαιδεύσουμε 100διαστατα word2vec embeddings με βάση τις προτάσεις του βηματος 9α.

In [126]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
path = get_tmpfile("word2vec.model")
model = Word2Vec(text, size=100, window=5, min_count=1)
model.save("word2vec.model")
model.train(text, total_examples=len(text), epochs=1000)
voc = model.wv.index2word
dim = model.wv.vector_size

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import numpy as np
def to_embeddings_Matrix(model):  
    embedding_matrix = np.zeros((len(model.wv.vocab), model.vector_size))
    word2idx = {}
    for i in range(len(model.wv.vocab)):
        embedding_matrix[i] = model.wv[model.wv.index2word[i]]
        word2idx[model.wv.index2word[i]] = i
    return embedding_matrix, model.wv.index2word, word2idx

  γ) Βρισκουμε 10 τυχαιες λεξεις απο το λεξικο και παραγουμε τις σημασειολογικα κοντινοτερες τους:

In [128]:
text1 = flatten(text)
Lexicon = unique_tokens(text1)
for i in range(0,10):
    word = random.choice(Lexicon)
    result, similarity = model.wv.most_similar(word,topn=1)[0][0],  model.wv.most_similar(word,topn=1)[0][1]
    print("Initial word: ", word, ", Similar-context word (According to Word2Vec embedding):", result, "\n Accuracy Achieved: ", similarity)
    

Initial word:  offered , Similar-context word (According to Word2Vec embedding): delicate 
 Accuracy Achieved:  0.4273199439048767
Initial word:  host , Similar-context word (According to Word2Vec embedding): expended 
 Accuracy Achieved:  0.4371470808982849
Initial word:  evident , Similar-context word (According to Word2Vec embedding): imprison 
 Accuracy Achieved:  0.4525837302207947
Initial word:  affection , Similar-context word (According to Word2Vec embedding): doyle 
 Accuracy Achieved:  0.4673263430595398
Initial word:  butted , Similar-context word (According to Word2Vec embedding): shawl 
 Accuracy Achieved:  0.4800434112548828
Initial word:  tomfoolery , Similar-context word (According to Word2Vec embedding): fleshy 
 Accuracy Achieved:  0.51698899269104
Initial word:  stalls , Similar-context word (According to Word2Vec embedding): imperturbably 
 Accuracy Achieved:  0.4985199272632599
Initial word:  papier , Similar-context word (According to Word2Vec embedding): eg 
 Acc

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Βήμα 10ο

  α) Εξάγουμε την πιθανότητα εμφάνισης κάθε token (λέξης) και την αποθηκεύουμε τη σε ένα λεξικό με key το token και value την πιθανότητα εμφάνισής του.

In [0]:
lines = read_from_file("corpora.txt")
text = []
Word_probability = {}
Totalwords = 0
for line in lines:
    text.append(tokenize(line))
for i in range(0,len(text)):
  for word in text[i]:
      if not word:
        continue
      Totalwords+= 1
      if Word_probability.get(word) == None:
        Word_probability.update({word:1})
      else:
        Word_probability[word] += 1
for key,value in Word_probability.items():
  Word_probability[key] = Word_probability[key]/Totalwords


 β) Εξάγουμε την πιθανότητα εμφάνισης κάθε χαρακτήρα και την αποθηκεύουμε τη σε ένα λεξικό με key το token και value την πιθανότητα εμφάνισής του.

In [0]:
lines = read_from_file("corpora.txt")
text = []
Letter_probability = {}
Totalletters = 0
for line in lines:
    text.append(tokenize(line))
for i in range(0,len(text)):
  for word in text[i]:
    for letter in range(0,len(word)):
      Totalletters+= 1
      if Letter_probability.get(word[letter]) == None:
        Letter_probability.update({word[letter]:1})
      else:
        Letter_probability.update({word[letter]:Letter_probability[word[letter]]+1})
for key,value in Letter_probability.items():
  Letter_probability[key] = Letter_probability[key]/Totalletters


#Βήμα 11ο

  α)Υπολογιζουμε την μεση τιμη των βαρων του word level, αλλα και του unigram level:

In [131]:
Wordw = 0
Distwords = 0
Letterw = 0
Distletters = 0
for (key,value) in Word_probability.items():
  Wordw -= np.log(value)
  Distwords += 1
for (key,value) in Letter_probability.items():
  Letterw -= np.log(value)
  Distletters += 1
Wordw = Wordw / Distwords
print(Distwords)
print(Wordw)
Letterw = Letterw / Distletters
print(Distletters)
print(Letterw)

11249
11.008959701031827
26
3.9208175099324643


  β) Αρχικα φτιαχνουμε μια συναρτηση για να τυπωνουμε σε αρχειο τις απαραιτητες πληροφοριες για να φτιαχτει το fst:

In [0]:
def weighted_fst(Alphabet,weight, filename = "text.txt", print_lines = False):
    f = open(filename, "w+")
    if (print_lines):
        for pair in Alphabet:
            for pair1 in Alphabet:
                if  (pair[0] == pair1[0] =="<epsilon>"):
                    continue 
                elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " 0")
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
                elif (pair[0]=='<epsilon>'):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight))
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight)+ "\n")
                elif (pair1[0]=='<epsilon>'):
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight))
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight)+ "\n")
                else:
                    print("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight))
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight)+ "\n")
        print("0")
        f.write("0")
    else:
        for pair in Alphabet:
            for pair1 in Alphabet:
                if  (pair[0] == pair1[0] =="<epsilon>"):
                    continue 
                elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
                elif (pair[0]=='<epsilon>'):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight)+ "\n")
                elif (pair1[0]=='<epsilon>'):
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight)+ "\n")
                else:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(weight)+ "\n")
        f.write("0")
    f.close()

Φτιαχνουμε τα δυο αρχεια που θα μας χρειαστουν για τους μετατροπεις:

In [0]:
lines = read_from_file("corpora.txt")
#print(len(lines))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
#print(len(text))
Lexicon = unique_tokens(text)
#print(len(Lexicon))
Alphabet = alphabet(Lexicon)
#print(len(Alphabet))
Alphabet = alphabet_indexing(Alphabet)
#print(Alphabet)
symbols_file(Alphabet,)
weighted_fst(Alphabet,Wordw, "Wordw.txt")
weighted_fst(Alphabet,Letterw, "Letterw.txt")

Και τωρα κανουμε compile τους μετατροπεις:

In [134]:
!fstcompile -isymbols=chars.syms -osymbols=chars.syms Wordw.txt > Wordw.fst
!fstinfo Wordw.fst
!fstcompile -isymbols=chars.syms -osymbols=chars.syms Letterw.txt > Letterw.fst
!fstinfo Letterw.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       1
# of arcs                                         728
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               26
# of output epsilons                              26
input label multiplicity                          26.9643
output label multiplicity                         26.9643
# of accessible states                            1
# of coaccessible states                          1
# of connected states                             1
# of connected components                         1
# of strongly conn components                     1
input matcher                 

# Βήμα 12ο

   α) Φτιαχνουμε 2 συναρτησεις για να δημιουργουμε τα αρχεια που ειναι απαραιτητα για τα  fst που μας ζητουνται:

In [0]:
def twelvtha(Lexicon, Alphabet, Word_probability, filename = "12a.txt"):
    for i in range(0, len(Alphabet)):
        temp = Alphabet[i]
        Alphabet[i] = [temp, i]
    f = open(filename, "w+")
    i = 2
    for word in Lexicon:
        for j in range(0, len(word)):
            if j==0:
                f.write("0" + " " + str(i) + " " + word[j] + " " + word[j] + " " + str(-1*np.log(Word_probability[word])) + "\n")
                i += 1
            elif (j== len(word) -1):
                f.write( str(i-1) + " " + "1" + " " + word[j] + " " + word[j] + " 0" + "\n")
                i +=1
            else:
                f.write( str(i-1) + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                i+=1
    f.write("1")
    f.close()

def twelvthb(Lexicon, Alphabet, Letter_probability, filename = "12b.txt"):
    for i in range(0, len(Alphabet)):
        temp = Alphabet[i]
        Alphabet[i] = [temp, i]
    f = open(filename, "w+")
    i = 2
    for word in Lexicon:
        for j in range(0, len(word)):
            if j==0:
                f.write("0" + " " + str(i) + " " + word[j] + " " + word[j] + " " + str(-1*np.log(Letter_probability[word[j]])) + "\n")
                i += 1
            elif (j== len(word) -1):
                f.write( str(i-1) + " " + "1" + " " + word[j] + " " + word[j] + " " + str(-1*np.log(Letter_probability[word[j]])) + "\n")
                i +=1
            else:
                f.write( str(i-1) + " " + str(i) + " " + word[j] + " " + word[j] + " " + str(-1*np.log(Letter_probability[word[j]])) + "\n")
                i+=1
    f.write("1")
    f.close()

Δημιουργουμε τα αρχεια που χρειαζομαστε:

In [0]:
lines = read_from_file("corpora.txt")
#print(len(lines))
text = []
for line in lines:
    text.append(tokenize(line))
text = flatten(text)
#print(len(text))
Lexicon = unique_tokens(text)
#print(len(Lexicon))
Alphabet = alphabet(Lexicon)
twelvtha(Lexicon, Alphabet,Word_probability,"Wordlevel.txt")
twelvthb(Lexicon, Alphabet,Letter_probability,"Unigram.txt")


  β) Καλουμε τις αναλογες συναρτησεις για να βελτιστοποιησουμε τα μοντελα μας:

In [137]:
!fstcompile -isymbols=chars.syms -osymbols=chars.syms Wordlevel.txt | fstrmepsilon | fstdeterminize | fstminimize > Wordlevel.fst
!fstinfo Wordlevel.fst
!fstcompile -isymbols=chars.syms -osymbols=chars.syms Unigram.txt | fstrmepsilon | fstdeterminize | fstminimize > Unigram.fst
!fstinfo Unigram.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       10295
# of arcs                                         18601
initial state                                     0
# of final states                                 1983
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          1
output label multiplicity                         1
# of accessible states                            10295
# of coaccessible states                          10295
# of connected states                             10295
# of connected components                         1
# of strongly conn components                     10295
input matcher      

#Βήμα 13ο

α)β)Για να κατασκευασουμε τους ορθογραφους για τα word-level & unigram-level γλωσσικα μοντελα καλουμε την fstcompose με τον word-level μετατροπεα:

In [138]:
! fstcompose Wordlevel.fst Wordw.fst > Wordlevellast.fst
! fstinfo Wordlevellast.fst
! fstcompose Unigram.fst Letterw.fst > Unigramlast.fst
! fstinfo Unigramlast.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       10295
# of arcs                                         769897
initial state                                     0
# of final states                                 1983
# of input/output epsilons                        0
# of input epsilons                               267670
# of output epsilons                              18601
input label multiplicity                          26.6523
output label multiplicity                         3.87038
# of accessible states                            10295
# of coaccessible states                          10295
# of connected states                             10295
# of connected components                         1
# of strongly conn components                     102

γ) Αρχικα καλουμε την fstcompose με το μοντελο sample.fst, και για τους δυο ορθογραφους:

In [0]:
Lexicon_sample = ["fuckery"]
sixth(Lexicon_sample, Alphabet, "sample.txt")

In [0]:
! fstcompile -isymbols=chars.syms -osymbols=chars.syms sample.txt > sample.fst
! fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait sample.fst  | dot -Tjpg >sample.jpg

In [141]:
! fstcompose  Wordlevellast.fst sample.fst > min_distance_Wordlevel.fst
! fstinfo min_distance_Wordlevel.fst



! fstcompose  Unigramlast.fst sample.fst > min_distance_Unigram.fst
! fstinfo min_distance_Unigram.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       82360
# of arcs                                         351080
initial state                                     0
# of final states                                 1983
# of input/output epsilons                        0
# of input epsilons                               72065
# of output epsilons                              148808
input label multiplicity                          1.74175
output label multiplicity                         3.70814
# of accessible states                            82360
# of coaccessible states                          82360
# of connected states                             82360
# of connected components                         1
# of strongly conn components                     823

Στην συνεχεια βρισκουμε την λεξη με την μικροτερη αποσταση απο την λεξη "cit" και για τους δυο ορθογραφους:

In [142]:
!fstshortestpath min_distance_Wordlevel.fst | fstrmepsilon > out_word.fst
!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait out_word.fst  | dot -Tjpg >out_word.jpg
!fstinfo out_word.fst



!fstshortestpath min_distance_Unigram.fst | fstrmepsilon > out_unigram.fst
!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait out_unigram.fst  | dot -Tjpg >out_unigram.jpg
!fstinfo out_unigram.fst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       8
# of arcs                                         7
initial state                                     7
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          1
output label multiplicity                         1
# of accessible states                            8
# of coaccessible states                          8
# of connected states                             8
# of connected components                         1
# of strongly conn components                     8
input matcher                                 

Αυτοι οι ορθογραφοι προσπαθουν να βρουν την λεξη που με ελαχιστον edit cost ερχεται πλησιεστερα στην λεξη εισοδου σε επιπεδο λεξεων και χαρακτηρων. Με αλλα λογια, ο ενας προσπαθει να ταιριαξει ολοκληρη την λεξη με βαση τις υπολοιπες λεξεις του κειμενου, ενω ο αλλος προσπαθει να ταιριαξει την λεξη ταιριαζοντας χαρακτηρες, ετσι ωστε να δημιουργησει μια λεξη που υπαρχει στο κειμενο. Ενα παραδειγμα αμφισημης διορθωσης ειναι η λεξη-εισοδος "fuckery", που με το μοντελο λεξεων διορθωνεται σε "mockery", ενω με το μοντελο unigram διορθωνεται σε "fury".

#Βήμα 14ο

  α) Το συνολο δεδομενων επαληθευσης ειναι στο αρχειο evaluation_data.txt


β) Ο κωδικας για να ελεγχουμε για καθε λεξη του συνολου δεδομενων επαληθευσης ειναι ο εξης:

In [143]:
lines = read_from_file("evaluation_data.txt")
text = []
dictionary = {}
for line in lines:
    text = []
    text.append(tokenize(line))
    for i in range (1,len(text[0])):
      dictionary.update({text[0][i]:text[0][0]})

print(dictionary)

{'contenpted': 'contented', 'contende': 'contented', 'contended': 'contented', 'contentid': 'contented', 'begining': 'beginning', 'problam': 'problem', 'proble': 'problem', 'promblem': 'problem', 'proplen': 'problem', 'dirven': 'driven', 'exstacy': 'ecstasy', 'ecstacy': 'ecstasy', 'guic': 'juice', 'juce': 'juice', 'jucie': 'juice', 'juise': 'juice', 'juse': 'juice', 'localy': 'locally', 'compair': 'compare', 'pronounciation': 'pronunciation', 'transportibility': 'transportability', 'miniscule': 'minuscule', 'independant': 'independent', 'aranged': 'arranged', 'arrainged': 'arranged', 'poartry': 'poetry', 'poertry': 'poetry', 'poetre': 'poetry', 'poety': 'poetry', 'powetry': 'poetry', 'leval': 'level', 'basicaly': 'basically', 'triangulaur': 'triangular', 'unexpcted': 'unexpected', 'unexpeted': 'unexpected', 'unexspected': 'unexpected', 'stanerdizing': 'standardizing', 'varable': 'variable', 'futher': 'further', 'monitering': 'monitoring', 'biscits': 'biscuits', 'biscutes': 'biscuits', 

In [144]:
for key in dictionary:
  value = dictionary[key]
  Lexicon_test = [key]
  f = open('new.txt', 'w+')
  f.truncate(0)
  f.close()
  fileopen = open("new.txt","w+")
  for i in range(0,len(key)):
    fileopen.write("%s " %str(i))
    fileopen.write(" %s " %str(i+1))
    fileopen.write(" %s " %key[i])
    fileopen.write(" %s " %key[i])
    fileopen.write("0\n")
  fileopen.write(str(len(key)))
  fileopen.close()
  sixth(Lexicon_test, Alphabet, "new.txt")
  !fstcompile -isymbols=chars.syms -osymbols=chars.syms new.txt > test.fst
  !fstcompose Wordlevellast.fst test.fst   > min_distance.fst
  print("Wrong word = " ,key,", actual word = ",value)
  print("What we got from the transducer =", end =" ")  
  !fstshortestpath min_distance.fst | fstrmepsilon | fsttopsort | fstprint -osymbols=chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n'
  print()


Wrong word =  contenpted , actual word =  contented
What we got from the transducer = contenpted
Wrong word =  contende , actual word =  contented
What we got from the transducer = contende
Wrong word =  contended , actual word =  contented
What we got from the transducer = contended
Wrong word =  contentid , actual word =  contented
What we got from the transducer = contentid
Wrong word =  begining , actual word =  beginning
What we got from the transducer = begining
Wrong word =  problam , actual word =  problem
What we got from the transducer = problam
Wrong word =  proble , actual word =  problem
What we got from the transducer = proble
Wrong word =  promblem , actual word =  problem
What we got from the transducer = promblem
Wrong word =  proplen , actual word =  problem
What we got from the transducer = proplen
Wrong word =  dirven , actual word =  driven
What we got from the transducer = dirven
Wrong word =  exstacy , actual word =  ecstasy
What we got from the transducer = exst

KeyboardInterrupt: ignored

In [0]:
for key in dictionary:
  value = dictionary[key]
  Lexicon_test = [key]
  f = open('new.txt', 'w+')
  f.truncate(0)
  f.close()
  fileopen = open("new.txt","w+")
  for i in range(0,len(key)):
    fileopen.write("%s " %str(i))
    fileopen.write(" %s " %str(i+1))
    fileopen.write(" %s " %key[i])
    fileopen.write(" %s " %key[i])
    fileopen.write("0\n")
  fileopen.write(str(len(key)))
  fileopen.close()
  sixth(Lexicon_test, Alphabet, "new.txt")
  !fstcompile -isymbols=chars.syms -osymbols=chars.syms new.txt > test.fst
  !fstcompose Unigramlast.fst  test.fst  > min_distance.fst
  print("Wrong word = " ,key,", actual word = ",value)
  print("What we got from the transducer =", end =" ")  
  !fstshortestpath min_distance.fst | fstrmepsilon | fsttopsort | fstprint -osymbols=chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n'
  print()
